In [1]:
import numpy as np
import pandas as pd
import statistics
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter, column_index_from_string
import random
from datetime import datetime
import seaborn as sns
import datetime
import os

from openpyxl import load_workbook
from openpyxl.utils import get_column_letter, column_index_from_string

pd.set_option('max_rows', 10000)
pd.set_option('max_columns', 100)

In [2]:
# избавляется от цифр в строках (если цифри - первое, что есть в строке)
# возвращает список со старыми и изменёнными названиями колонок

def kill_numbers(df):
    new_cols = []
    for col_name in df.columns:
        if col_name == 'Предоставлен' or col_name == 'Submitted':
            new_cols.append('Дата')
        elif col_name.split()[0][0].isnumeric():
            new_cols.append(' '.join(col_name.split()[1:]))
        else:
            new_cols.append(col_name)
    df.columns = new_cols
    return df

In [3]:
def take_forms(forms): # принимает список файлов, возвращает 3 названия. Файлы должны начинаться с номера формы
                       # и содержать точку с пробелом после неё (прим: "47. ")
    for _ in forms:
        if '47. ' in _:
            m = _
        elif '49. ' in _:
            k = _
        elif '50. ' in _:
            r = _
    return m, k, r

In [4]:
def repair_date(df):
    if df['Дата'].max()[-2:] != df['Дата'].min()[-2:]:
        df['Дата'][df['Дата']==df['Дата'].max()] = df['Дата'].max()[:8] + df['Дата'].min()[-2:]
    return df

In [9]:
# current date - 28 days
str(datetime.datetime.today() - datetime.timedelta(28)).split(' ')[0]

'2022-04-06'

In [6]:
# меняем директорию на рабочую, смотрим на все файлы в директории
os.chdir(r'C:\Users\zhilapv\Desktop\Четверговый отчет\31.03.2022 - 28.04.2022')
forms_list = os.listdir()
forms_list

In [8]:
# в переменные m, k, r закидываем нужные файлы с помощью take_forms()
m, k, r = take_forms(os.listdir())

# формируем три соответствующих датасета, предварительно обработанных с помощью kill_numbers()
df_m, df_k, df_r = kill_numbers(pd.read_excel(m)).loc[:, 'Федеральный округ':'Дата'],\
                   kill_numbers(pd.read_excel(k)).loc[:, 'Федеральный округ':'Дата'],\
                   kill_numbers(pd.read_excel(r)).loc[:, 'Федеральный округ':'Дата']

# добавляем в каждый датафрейм две новые колонки
df_m['Тип организации'] = 'Медицинские'
df_m['Субъект и организация'] = df_m['Наименование организации']
df_k['Тип организации'] = 'Коммерция'
df_k['Субъект и организация'] = df_k['Субъект РФ'] + '. ' + df_k['Наименование организации']
df_r['Тип организации'] = 'Роспотребнадзор'
df_r['Субъект и организация'] = df_r['Субъект РФ'] + '. ' + df_r['Наименование организации']

# формируем новый датафрейм из трёх предыдущих
df = pd.concat([df_m, df_k])
df = pd.concat([df, df_r])

# фильтруем датафрейм
df = df[df['Количество проведенных исследований']>0]
#df = df[df['Выберите тип исследований, о которых вы хотите представить отчёт']!='Антитела']

# ещё одна колонка
df['Процент'] = df['Количество выявленных положительных результатов'] / df['Количество проведенных исследований']
df = df[df['Процент']<=1]

# оставляем только дату, время отлетает
df['Дата'] = df['Дата'].apply(lambda x: x.split(' ')[0])

df.head()

,Федеральный округ,Unnamed: 5,Субъект РФ,Наименование организации,ОГРН,"Количество сотрудников лаборатории, имеющих допуск и проводящих исследования","Максимальное суточное количество сотрудников лаборатории, фактически участвовавших в проведении исследований за прошедшую неделю",Наименования используемых ПЦР тест-систем,Наименования используемых тест-систем на антиген,Серия (партия) используемых тест-систем,Количество проведенных исследований,Количество выявленных положительных результатов,Остатки неиспользованных тест-систем,Количество обследованных людей,Сведения о проведении лабораторных исследований на наличие антител > Антитела IgG > Количество проведенных исследований,Сведения о проведении лабораторных исследований на наличие антител > Антитела IgG > Число положительных результатов,Сведения о проведении лабораторных исследований на наличие антител > Антитела IgM > Количество проведенных исследований,Сведения о проведении лабораторных исследований на наличие антител > Антитела IgM > Число положительных результатов,Сведения о проведении лабораторных исследований на наличие антител > Суммарные антитела IgG+IgM > Количество проведенных исследований,Сведения о проведении лабораторных исследований на наличие антител > Суммарные антитела IgG+IgM > Число положительных результатов,e-mail,"Выберите тип исследований, о которых вы хотите представить отчёт (лог)",Дата,Тип организации,Субъект и организация,"Выберите тип исследований, о которых вы хотите представить отчёт(лог)",Процент
0,Северо-Западный федеральный округ,NaN,г. Санкт-Петербург,г. Санкт-Петербург.СПБ ГБУЗ Детская городская ...,1.027809e+12,6.0,6.0,COVID-2019 Amp (ФБУН НИИ эпидемиологии и микро...,NaN,Lot 001,125.0,10.0,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ПЦР,2022-04-28,Медицинские,г. Санкт-Петербург.СПБ ГБУЗ Детская городская ...,NaN,0.080000
1,Северо-Западный федеральный округ,NaN,Архангельская область,Архангельская область. ГБУЗ АО ''Новодвинская ...,1.022901e+12,2.0,2.0,NaN,Набор реагентов ''Иммунохроматографический экс...,003,1.0,0.0,4329.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,labooi@fbuz29.rospotrebnadzor.ru,Антиген,2022-04-28,Медицинские,Архангельская область. ГБУЗ АО ''Новодвинская ...,NaN,0.000000
2,Северо-Западный федеральный округ,NaN,Новгородская область,Новгородская область. ГОБУЗ ''Центральная горо...,1.025301e+12,2.0,2.0,NaN,Экспресс-антиген SARS-CoV-2-ИХА (ООО ''РАПИД Б...,LSPZ20K01,277.0,28.0,34909.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,victoria942@mail.ru,Антиген,2022-04-28,Медицинские,Новгородская область. ГОБУЗ ''Центральная горо...,NaN,0.101083
3,Северо-Западный федеральный округ,NaN,Архангельская область,Архангельская область. ГБУЗ АО ''Мирнинская це...,1.022901e+12,2.0,2.0,NaN,Тест-система для качественного выявления антиг...,20112,23.0,5.0,2625.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,labooi@fbuz29.rospotrebnadzor.ru,Антиген,2022-04-28,Медицинские,Архангельская область. ГБУЗ АО ''Мирнинская це...,NaN,0.217391
4,Северо-Западный федеральный округ,NaN,Архангельская область,Архангельская область. ГБУЗ АО ''Коряжемская г...,1.122904e+12,2.0,2.0,NaN,Тест-система для качественного выявления антиг...,20112,32.0,0.0,1194.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,labooi@fbuz29.rospotrebnadzor.ru,Антиген,2022-04-28,Медицинские,Архангельская область. ГБУЗ АО ''Коряжемская г...,NaN,0.000000


In [9]:
# сохраняем эксельку, дальше работаем там
df.to_excel('Общий свод 28.04.2022.xlsx', float_format='%.2f', index=False)